In [55]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

whole_df = pd.read_csv('../data/clean_df.csv', index_col=[0])

In [56]:
whole_df.loc[whole_df['behavior'].isin(["bra","cam","corr","tre"]),'macro_bhv'] = 'moving'
whole_df.loc[whole_df['behavior'].isin(["sen","quad","tum"]),'macro_bhv'] = 'resting'
whole_df.loc[whole_df['behavior'].isin(["alim","forr"]),'macro_bhv'] = 'feeding'
whole_df.loc[whole_df['behavior'].isin(["groo","hur","ras","chup"]),'macro_bhv'] = 'grooming'
whole_df.loc[whole_df['behavior'].isin(["jsol","jexp","jacrob"]),'macro_bhv'] = 'playing'
whole_df.loc[whole_df['behavior'].isin(["ins"]),'macro_bhv'] = 'exploring'
whole_df.loc[whole_df['behavior'].isin(["sex"]),'macro_bhv'] = 'sex'
whole_df.loc[whole_df['behavior'].isin(["con","marc","huir"]),'macro_bhv'] = 'agonistic'
whole_df.loc[whole_df['behavior'].isin(["lucha","perj"]),'macro_bhv'] = 'social-playing'
whole_df.loc[whole_df['behavior'].isin(["pub"]),'macro_bhv'] = 'interacting with public'
whole_df.loc[whole_df['behavior'].isin(["jun","aprox","trans","toca","solj","alogroo"]),'macro_bhv'] = 'social'


In [57]:
whole_df.head()

,date,subject,period,reg,behavior,receptor,duration,macro_bhv
0,2019-04-15,Karl,pregame,1,cam,[],15.0,moving
1,2019-04-15,Karl,pregame,1,forr,[],34.0,feeding
2,2019-04-15,Karl,pregame,1,sen,[],52.0,resting
3,2019-04-15,Karl,pregame,1,cam,[],5.0,moving
4,2019-04-15,Karl,pregame,1,sen,[],34.0,resting


In [5]:
whole_df.to_csv('../data/clean_df.csv')

In [177]:

a= go.Histogram(x=whole_df[whole_df['behavior']=='cam']['duration'])
b= go.Histogram(x=whole_df[whole_df['behavior']=='alogroo']['duration'])
fig = make_subplots(rows=2, cols=2)
fig.append_trace(a, 1, 1)
fig.append_trace(b, 1, 2)
fig.update_layout({
    'plot_bgcolor': 'rgba(21, 47, 18, 255)',
    'paper_bgcolor': 'rgba(21, 47, 18, 255)'
})
fig.show()

In [179]:
import math
beh=(whole_df['behavior'].unique())
def behavior_histogram(behavior):
    return go.Histogram(x=whole_df[whole_df['behavior']==behavior]['duration'],
    name=behavior,
    marker={'color':'#C5D845'})
fig = make_subplots(rows=4, cols=8, subplot_titles=beh, vertical_spacing=0.2)
for b in range(len(beh)):
    row = math.floor(b/8+1)
    col = b%8+1
    fig.append_trace(behavior_histogram(beh[b]), row, col)
fig.update_layout({'height':600,
                   'showlegend':False, 
                   'plot_bgcolor': 'rgba(21, 47, 18, 255)',
                   'paper_bgcolor': 'rgba(21, 47, 18, 255)',
                   'font':{'color': 'rgba(230,230,230,255)'},
                   'yaxis':{'showgrid':False}
                   })
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False, showgrid=False)
fig.show()

In [11]:
whole_df['subject'].unique()

array(['Karl', 'Locky', 'Jawie', 'Storma', 'Popo', 'Hadia'], dtype=object)

In [53]:
ml_df = pd.read_csv('../data/machine_learning.csv')
ml_df[:].sum(axis=1)

0       612.0
1       603.0
2       544.0
3      1005.0
4       546.0
        ...  
533    1135.0
534    1141.0
535    1137.0
536    1138.0
537    1135.0
Length: 538, dtype: float64

In [180]:
grouped_duration=whole_df.groupby(['date','reg', 'subject'])['duration'].sum().reset_index()
grouped_duration[grouped_duration['duration']>650].sort_values(by=['subject', 'date'])

,date,reg,subject,duration
24,2019-04-17,1,Hadia,660.0
84,2019-04-24,1,Hadia,677.0
205,2019-05-23,1,Jawie,665.0
265,2019-05-30,2,Jawie,651.0
388,2019-06-27,2,Jawie,652.0
302,2019-06-07,2,Karl,665.0
384,2019-06-27,1,Locky,668.0


In [175]:
date='2019-04-24'
sub='Hadia'
reg=1

whole_df[(whole_df['date']==date)&(whole_df['subject']==sub)&(whole_df['reg']==reg)]

,date,subject,period,reg,behavior,receptor,duration,macro_bhv
6680,2019-04-24,Hadia,pregame,1,jun,['Storma'],55.0,social
6681,2019-04-24,Hadia,pregame,1,tre,[],44.0,moving
6682,2019-04-24,Hadia,pregame,1,jexp,[],48.0,playing
6683,2019-04-24,Hadia,pregame,1,bra,[],6.0,moving
6684,2019-04-24,Hadia,pregame,1,jun,['Storma'],31.0,social
6685,2019-04-24,Hadia,pregame,1,tre,[],12.0,moving
6686,2019-04-24,Hadia,pregame,1,forr,[],31.0,feeding
6687,2019-04-24,Hadia,pregame,1,tre,[],8.0,moving
6688,2019-04-24,Hadia,pregame,1,forr,[],10.0,feeding
6689,2019-04-24,Hadia,pregame,1,bra,[],13.0,moving


In [166]:
whole_df.loc[749,'duration']=213-166


In [171]:
whole_df.drop(5559, inplace=True)

In [182]:
px.box(whole_df[(whole_df['behavior']=='sex')], y='duration', width=400)

In [181]:
whole_df.to_csv('../data/clean_df.csv')